In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
#import warnings
#warnings.filterwarnings("ignore")
import janestreet

In [ ]:
%%time
train = pd.read_csv('/kaggle/input/jane-street-market-prediction/train.csv')
print('train shape is {}'.format(train.shape))

In [ ]:
def metric (df, actions):
    buy = actions>0.5
    gains = pd.DataFrame({'date': df['date'][buy], 'gain': df['weight'][buy] * df['resp'][buy]})
    p = gains.groupby('date').sum().to_numpy()
    s = p.sum()
    return np.clip(s / np.sqrt((p*p).sum() * p.shape[0] / 250), a_min=0, a_max=6) * s
model = lambda df: (df['feature_34']>0)&(df['feature_43']>0)
print(metric (train, model(train)))

In [ ]:
%%time
env = janestreet.make_env() # initialize the environment
iter_test = env.iter_test() # an iterator which loops over the test set
for (test_df, prediction_df) in iter_test:
    prediction_df.action = model(test_df).astype(int)
    env.predict(prediction_df)